In [1]:
import sys
import sys
sys.path.append('../../')
from omnicell.constants import *

from omnicell.models.gears.gears import GEARS
import omnicell.models.gears.pertdata as pertdata_new
import omnicell.models.gears.pertdata_old as pertdata_old
from omnicell.data.catalogue import Catalogue
from omnicell.models.gears.pertdata import PertData
from omnicell.data.catalogue import Catalogue
from omnicell.models.gears.gears import GEARS
import scanpy as sc
import torch
import numpy as np


In [2]:


DATASET_NAME = 'satija_IFNB_raw'
dd = Catalogue.get_dataset_details(DATASET_NAME)
adata = sc.read(dd.path)

adata = adata[adata.obs[dd.cell_key] == 'A549']

#random selection of 20 perts + control
perts = np.random.choice(adata.obs[dd.pert_key].unique(), 10, replace=False)
perts = set(perts).union(set([dd.control]))
adata = adata[adata.obs[dd.pert_key].isin(perts)]


adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,sample,bc1_well,bc2_well,bc3_well,percent.mito,cell_type,pathway,RNA_snn_res.0.9,seurat_clusters,sample_ID,Batch_info,guide,gene,mixscale_score
06_28_67_1_1_1_1_1_1_1_1_1,6,8999,3854,A549_IFNB,A6,C4,F7,9.189910,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg8,NT,0.000000
06_27_93_1_1_1_1_1_1_1_1_1,6,8384,3600,A549_IFNB,A6,C3,H9,3.268130,A549,IFNB,15.0,15.0,sample_1,Rep1,STAT5Ag1,STAT5A,0.377627
07_79_66_1_1_1_1_1_1_1_1_1,7,7873,3300,A549_IFNB,A7,G7,F6,6.236505,A549,IFNB,15.0,15.0,sample_1,Rep1,RNF114g3,RNF114,-1.004491
06_24_45_1_1_1_1_1_1_1_1_1,6,7015,3393,A549_IFNB,A6,B12,D9,6.557377,A549,IFNB,15.0,15.0,sample_1,Rep1,TRIM22g1,TRIM22,-0.666294
06_84_11_1_1_1_1_1_1_1_1_1,6,6593,3475,A549_IFNB,A6,G12,A11,4.641286,A549,IFNB,15.0,15.0,sample_1,Rep1,STAT5Ag2,STAT5A,1.337676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07_77_55_2_2,7,2087,1451,A549_IFNB,A7,G5,E7,2.635362,A549,IFNB,NaN,NaN,sample_16,Rep2,MAP3K14g2,MAP3K14,-0.997436
08_50_26_2_2,8,2064,1223,A549_IFNB,A8,E2,C2,3.536822,A549,IFNB,NaN,NaN,sample_16,Rep2,RNF114g3,RNF114,0.396394
08_10_10_2_2,8,1961,1384,A549_IFNB,A8,A10,A10,1.529832,A549,IFNB,NaN,NaN,sample_16,Rep2,STAT2g2,STAT2,3.465757
05_46_58_2_2,5,1555,1024,A549_IFNB,A5,D10,E10,12.347267,A549,IFNB,NaN,NaN,sample_16,Rep2,STAT2g3,STAT2,2.802478


In [3]:

adata.obs["condition"] = adata.obs[dd.pert_key]
perts = [p for p in adata.obs["condition"].unique() if p != dd.control]
adata.obs["condition"] = adata.obs["condition"].replace({dd.control:"ctrl"})
adata.obs["condition"] = adata.obs["condition"].replace({p:p+"+ctrl" for p in perts})
adata.var["gene_name"] = adata.var_names




In [4]:



print(f"Data loaded from {dd.path}")
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)
print(f"Data normalized and log transformed")

device = 'cuda' if torch.cuda.is_available() else 'cpu'



Data loaded from /orcd/data/omarabu/001/Omnicell_datasets/satija_IFNB_raw/Seurat_IFNB.h5ad
Data normalized and log transformed


In [ ]:


pert_data = pertdata_old.PertData('./data') # specific saved folder
print(f"Saving data in ./data")

pert_data.new_data_process(dataset_name = DATASET_NAME, adata = adata, skip_calc_de=False) # specific dataset name and adata object
print(f"Data processed and saved in {pert_data.data_path}")
pert_data.prepare_split(split = 'no_test', seed = 1) # get data split with seed
print(f"Data split with seed 1")
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader
print(f"Data loader prepared")


In [ ]:



gears_model = GEARS(pert_data, device = device, 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')

gears_model.model_initialize(hidden_size = 64)

gears_model.tunable_parameters()



In [ ]:

print(f"Training model")
gears_model.train(epochs = 1, lr = 1e-3)


gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')


res = gears_model.predict([['RPL15']])

np.savez('res.npz', res)



In [5]:
pert_data_new = pertdata_new.PertData('./data_new') # specific saved folder


print(f"Saving data in ./data")

pert_data_new.new_data_process(dataset_name = DATASET_NAME, adata = adata, skip_calc_de=True) # specific dataset name and adata object
print(f"Data processed and saved in {pert_data_new.data_path}")
pert_data_new.prepare_split(split = 'no_test', seed = 1) # get data split with seed
print(f"Data split with seed 1")
pert_data_new.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader
print(f"Data loader prepared")


Downloading...
100%|██████████| 9.46M/9.46M [00:00<00:00, 12.9MiB/s]


Saving data in ./data
Starting new data processing for satija_IFNB_raw
Getting DE Genes
Got DE genes for satija_ifnb_raw: 0.26 seconds


Downloading...
100%|██████████| 559k/559k [00:00<00:00, 2.13MiB/s]


Test
Creating pyg object for each cell in the data...
Creating dataset file...


  9%|▉         | 1/11 [00:00<00:01,  7.35it/s]

...

Creating 436 cell graphs for IRF7+ctrl: 0.03 seconds
Cell graph dataset for IRF7+ctrl created, first element: Data(x=[5000, 1], y=[1, 5000], pert_idx=[1], de_idx=[20], pert='IRF7+ctrl')
create_cell_graph_dataset call duration: IRF7+ctrl: 0.06 seconds
Ctrl data shape (415, 5000), adata shape (415, 5000)
Creating 415 cell graphs for NFE2L3+ctrl, Xs[0] shape (5000,), ys[0] shape (5000,)


100%|██████████| 11/11 [00:01<00:00,  9.96it/s]

Creating 415 cell graphs for NFE2L3+ctrl: 0.24 seconds
Cell graph dataset for NFE2L3+ctrl created, first element: Data(x=[5000, 1], y=[1, 5000], pert_idx=[1], de_idx=[20], pert='NFE2L3+ctrl')
create_cell_graph_dataset call duration: NFE2L3+ctrl: 0.26 seconds
MAP3K14+ctrl
Ctrl data shape (407, 5000), adata shape (407, 5000)
Creating 407 cell graphs for MAP3K14+ctrl, Xs[0] shape (5000,), ys[0] shape (5000,)
Creating 407 cell graphs for MAP3K14+ctrl: 0.03 seconds
Cell graph dataset for MAP3K14+ctrl created, first element: Data(x=[5000, 1], y=[1, 5000], pert_idx=[1], de_idx=[20], pert='MAP3K14+ctrl')
create_cell_graph_dataset call duration: MAP3K14+ctrl: 0.06 seconds
Done!
Saving new dataset pyg object at ./data_new/satija_ifnb_raw/data_pyg/cell_graphs.pkl



Creating new splits....
Saving new splits at ./data_new/satija_ifnb_raw/splits/satija_ifnb_raw_no_test_1_0.75.pkl
Done!
Creating dataloaders....
Done!


Done!
Data processed and saved in ./data_new
Data split with seed 1
Data loader prepared


In [6]:

gears_model = GEARS(pert_data_new, device = device, 
                        weight_bias_track = False, 
                        proj_name = 'pertnet_new', 
                        exp_name = 'pertnet_new')

gears_model.model_initialize(hidden_size = 64)

gears_model.tunable_parameters()




Downloading...
100%|██████████| 60.7M/60.7M [00:26<00:00, 2.26MiB/s]
Extracting tar file...
Done!


{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

In [7]:
next(iter(gears_model.dataloader["train_loader"]))

DataBatch(x=[160000, 1], y=[32, 5000], pert_idx=[32], de_idx=[32], pert=[32], batch=[160000], ptr=[33])

In [8]:

print(f"Training model")
gears_model.train(epochs = 1, lr = 1e-3)




gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')


res = gears_model.predict([['RPL15']])

np.savez('res.npz', res)


Start Training...


Training model
Dataloader element: DataBatch(x=[160000, 1], y=[32, 5000], pert_idx=[32], de_idx=[32], pert=[32], batch=[160000], ptr=[33])


Epoch 1 Step 1 Train Loss: 0.3383
Epoch 1 Step 51 Train Loss: 0.3553
Epoch 1 Step 101 Train Loss: 0.3153
Epoch 1 Step 151 Train Loss: 0.2760
Epoch 1: Train Overall MSE: 0.0014 Validation Overall MSE: 0.0009. 
Train Top 20 DE MSE: 0.0341 Validation Top 20 DE MSE: 0.0035. 
Done!
Done! No test dataloader detected.
